In [1]:
import requests
# 构造请求头,模拟浏览器
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive'
}
url="https://data1.library.sh.cn/webapi/zoutaofen/getHumanYearListInterface?key=692e3a8a6818cceb3c8977664c798dac98e81d05&stime=1900-01-01&etime=2000-01-01&keyword=邹韬奋"

# 发送GET请求
response = requests.get(url, headers=headers)

# 检查请求是否成功
if response.status_code == 200:
    # 获取响应数据
    # data = response.text  # 文本格式
    data = response.json()  # JSON格式
    print("数据获取成功!")
    print(data)
else:
    print(f"请求失败,状态码: {response.status_code}")


数据获取成功!
{'page': 1, 'records': 24, 'extend': {'sub': '本名恩润，乳名荫书，曾用名李晋卿。祖籍江西省鹰潭市余江区潢溪镇渡口村委会沙塘村，1895年出生于福建永安。近代中国记者和出版家。 \n1922年在黄炎培等创办的中华职业教育社任编辑部主任，开始从事教育和编辑工作。1926年接任《生活周刊》主编，以犀利之笔，力主正义舆论，抨击黑暗势力。 \n1931年“九·一八”事变后，邹韬奋在上海全身心投入抗日救亡运动。1935年12月，他与沈钧儒、厉麟似等人组织成立了上海文化界救国会。1936年11月，国民党为了扑灭国内的抗日烈火，逮捕了正在领导抗日救亡运动的救国会领导人沈钧儒、邹韬奋等七人，酿成七君子事件，遭到全国人民，包括宋庆龄、何香凝等社会名流的强烈反对。邹韬奋出狱后辗转重庆、汉口、香港继续开展爱国救亡工作。\n1943年因患耳癌秘密返沪就医。1944年7月24日不幸逝世于上海医院（今上海市第二人民医院），终年49岁。\n1956年上海市政府出资修复邹韬奋的故居，并在隔壁建立了韬奋纪念馆。2009年邹韬奋被评为100位为新中国成立作出突出贡献的英雄模范之一。.', 'stimer': '1895-11-5', 'sex': '男', 'wordname': '邹韬奋', 'yearno': '', 'stime': '1895-11-05 00:00:00', 'pic': '/Upload/image/20230628124722_4u.jpg', 'type': 1, 'etimer': '1944-7-24', 'userid': 'superadmin', 'retime': '2023-06-28 12:47:24', 'penname': '韬奋', 'familyID': '', 'books': '', 'feature': '', 'appraise': '伟大的爱国者、卓越的文化战士、杰出的出版家和新闻记者、创始人之一', 'enable': 1, 'etime': '1944-07-24 00:00:00', 'name': '邹韬奋', 'ID': 12, 'orgs': '生活书店', 'namer': '邹恩润,李晋卿', 'bornaddress': '福建', 'events': ''}

In [10]:
##尝试使用函数解析json，效果不好，使用后文的第三方网站吧
import json
import itertools
def get_structure_key(data):
    """
    生成一个可哈希的键来代表JSON对象的直接结构。
    用于对列表中的连续相似项进行分组。
    """
    if isinstance(data, dict):
        # 字典的结构由其键定义。
        return ('dict', tuple(sorted(data.keys())))
    if isinstance(data, list):
        # 列表的结构由其长度定义（这是一个简化，但在很多情况下有效）。
        return ('list', len(data))
    else:
        # 基本类型的结构由其类型本身定义。
        return ('primitive', type(data))


def parse_json_structure_to_lines(data, indent=0):
    """
    递归遍历JSON数据，并返回一个表示其结构的字符串列表。
    """
    indent_str = '  ' * indent
    lines = []

    if isinstance(data, dict):
        if not data:
            return [indent_str + '{}']
        lines.append(indent_str + '{')
        for key, value in data.items():
            value_lines = parse_json_structure_to_lines(value, indent + 1)
            # 如果值是基本类型或空集合，它只有一行。
            # 我们可以将其与键合并到同一行。
            is_simple = (
                not isinstance(value, (dict, list)) or
                not value
            )
            if is_simple and value_lines:
                lines.append(f'{indent_str}  "{key}": {value_lines[0].lstrip()}')
            else:
                lines.append(f'{indent_str}  "{key}":')
                lines.extend(value_lines)
        lines.append(indent_str + '}')
        return lines

    elif isinstance(data, list):
        if not data:
            return [indent_str + '[]']
        lines.append(indent_str + '[')

        # 对结构相同的连续项进行分组。
        structure_groups = [list(g) for g in itertools.groupby(data, key=get_structure_key)]

        for group in structure_groups:
            # 获取组中第一项的结构。
            item_lines = parse_json_structure_to_lines(group[0], indent + 1)
            lines.extend(item_lines)

            # 如果组中有更多项，则添加摘要行。
            if len(group) > 1:
                lines.append(f'{indent_str}  ... ({len(group) - 1} more item(s) with the same structure)')

        lines.append(indent_str + ']')
        return lines

    else:
        # 对于基本类型，使用ensure_ascii=False来保持中文字符
        return [indent_str + json.dumps(data, ensure_ascii=False)]


def analyze_json_and_save(data: dict, output_filepath: str):
    """
    将JSON字典解析为其结构树，并将结果保存到文件中。
    在同一层次的列表中，结构相同的连续元素将只显示第一个，
    其余的将由一条摘要行代替。

    :param data: 包含JSON数据的字典。
    :param output_filepath: 输出 .txt 文件的路径。
    """
    structure_lines = parse_json_structure_to_lines(data)
    result = '\n'.join(structure_lines)

    with open(output_filepath, 'w', encoding='utf-8') as f:
        f.write(result)

    print(f"JSON结构分析完成，结果已保存至 '{output_filepath}'")
    print("\n--- 文件内容预览 ---\n")
    print(result)

In [12]:
analyze_json_and_save(data,'./人物生平结构.txt')

JSON结构分析完成，结果已保存至 './人物生平结构.txt'

--- 文件内容预览 ---

{
  "page": 1
  "records": 24
  "extend":
  {
    "sub": "本名恩润，乳名荫书，曾用名李晋卿。祖籍江西省鹰潭市余江区潢溪镇渡口村委会沙塘村，1895年出生于福建永安。近代中国记者和出版家。 \n1922年在黄炎培等创办的中华职业教育社任编辑部主任，开始从事教育和编辑工作。1926年接任《生活周刊》主编，以犀利之笔，力主正义舆论，抨击黑暗势力。 \n1931年“九·一八”事变后，邹韬奋在上海全身心投入抗日救亡运动。1935年12月，他与沈钧儒、厉麟似等人组织成立了上海文化界救国会。1936年11月，国民党为了扑灭国内的抗日烈火，逮捕了正在领导抗日救亡运动的救国会领导人沈钧儒、邹韬奋等七人，酿成七君子事件，遭到全国人民，包括宋庆龄、何香凝等社会名流的强烈反对。邹韬奋出狱后辗转重庆、汉口、香港继续开展爱国救亡工作。\n1943年因患耳癌秘密返沪就医。1944年7月24日不幸逝世于上海医院（今上海市第二人民医院），终年49岁。\n1956年上海市政府出资修复邹韬奋的故居，并在隔壁建立了韬奋纪念馆。2009年邹韬奋被评为100位为新中国成立作出突出贡献的英雄模范之一。."
    "stimer": "1895-11-5"
    "sex": "男"
    "wordname": "邹韬奋"
    "yearno": ""
    "stime": "1895-11-05 00:00:00"
    "pic": "/Upload/image/20230628124722_4u.jpg"
    "type": 1
    "etimer": "1944-7-24"
    "userid": "superadmin"
    "retime": "2023-06-28 12:47:24"
    "penname": "韬奋"
    "familyID": ""
    "books": ""
    "feature": ""
    "appraise": "伟大的爱国者、卓越的文化战士、杰出的出版家和新闻记者、创始人之一"
    "enable": 1
    "etime": "1944-07-24 0

In [13]:
#将dict转为json字符串
json_str = json.dumps(
            data,
            ensure_ascii=False,  # 确保中文字符正确显示
            indent=0,       # 设置缩进
            sort_keys=True       # 按键排序，使输出更稳定
        )
#保存为.json文件
with open('./生平.json', 'w', encoding='utf-8') as f:
            f.write(json_str)
#交给https://json4u.cn/editor网站可视化处理

{'data': [{'items': [{'child': [],
     'data': 'item',
     'id': 4,
     'pic': '',
     'redate': '1908-01-01 00:00:00',
     'redater': '1908年',
     'sub': '<p>邹韬奋随父亲同去余江奔丧，并扶母亲灵柩同往，并在老家居住四个月。</p>',
     'text': '',
     'type': 0}],
   'year': 1908},
  {'items': [{'child': [],
     'data': 'item',
     'id': 58,
     'pic': '',
     'redate': '1923-07-10 00:00:00',
     'redater': '1923-7-10',
     'sub': '<p>中华职业学校与东献学合作，在暑期学校课程中加入职业教育一组内容，由黄炎培讲职业教育概论，朱经农、杨鄂联讲职业教育课程，廖世承、邹韬奋年谱长编邹恩润讲职业知能测试报告。</p><p>办事部门也暂移南京，上午照常办公，下午列席讲堂，傍晚游览名胜。 午后，邹恩润讲职业知能测验之性质范围及历史，此课程测验二门，于讲演毕后再行讨论。</p>',
     'text': '',
     'type': 0}],
   'year': 1923},
  {'items': [{'child': [],
     'data': 'item',
     'id': 92,
     'pic': '',
     'redate': '1926-12-01 00:00:00',
     'redater': '1926-12',
     'sub': '<p>备考：“邹韬奋先生在上海创办《生活》周刊后，曾多次到江西南昌心远中学与在校师生联络，请母校师生帮助宣传和推销《生活》周刊。 章瑞麟虽年近九旬，对此仍印象深刻。”</p>',
     'text': '',
     'type': 0}],
   'year': 1926},
  {'items': [{'child': [],
     'data': 'item',
     'id': 